# Technical-Analysis-Indicators Bokeh Charting

Code from https://github.com/rebekah999/Technical-Analysis-Indicators repository by Rebekah Fowler

Variable "data" is Five yrs of Daily historical Data from Yahoo Finance in CSV format data can be downloaded here: https://finance.yahoo.com/quote/AA/history?period1=1432080000&period2=1589932800&interval=1d&filter=history&frequency=1d

Charting with Bokeh library https://docs.bokeh.org/en/latest/

In [1]:
# basic imports for overall functionality
import numpy as np 
import pandas as pd 

# imports for graphs and charts
from bokeh.plotting import figure, show, output_file
from bokeh.models import ColumnDataSource, HoverTool, CDSView, BooleanFilter

#imports for layouts
from bokeh.layouts import column, row, gridplot

In [2]:
def dataprep(file, limit = 0):
    df = pd.read_csv(file, parse_dates = True)
    df["Date"] = pd.to_datetime(df["Date"])
    if limit != 0:
        return df.tail(limit)
    else:
        return df
    
def bb_source(df):
    source = ColumnDataSource(bb_candle_plotprep(df))
    return source

hex_color_green = "#38f93b"
hex_color_red = "#f9434e"

In [3]:
def def_ha(df):
    ha = pd.DataFrame()

    ha["Close"] = (df["Open"] + df["High"] + df["Low"] + df["Close"])/ 4
    ha["Open"]  = ((df["Open"] + df["Close"])/ 2).shift(1)
    ha["High"] = df[["High", "Open", "Close"]].max(axis = 1)
    ha["Low"] = df[["Low", "Open", "Close"]].min(axis = 1)
    ha["Date"] = df["Date"]
    return ha

In [4]:
def bb_candle_plotprep(df, period = 20, std = 2):
    

    bb = pd.DataFrame().reindex(df.index)
    df_add = df.loc[:, ["Open", "High", "Low", "Close"]].mean(axis = 1)
    rolling_mean = df_add.rolling(window = period).mean()
    rolling_std = df_add.rolling(window = period).std()
    bb["Upper"] = rolling_mean.add(rolling_std * std)
    bb["Middle"] = rolling_mean
    bb["Lower"] = rolling_mean.subtract(rolling_std * std)

    bb["Open"] = df["Open"].shift(1)
    bb["High"] = df["High"].shift(1)
    bb["Low"] = df["Low"].shift(1)
    bb["Close"] = df["Close"].shift(1)
    
    bb["Date"] = df["Date"]
    return bb

In [5]:
def reg_ha_bb_candle_plot(file, w = 12*60*60*1000, of_title = "AAPL.html", f_title = "AAPL"):
    prep = dataprep(file)
    
    df = bb_candle_plotprep(prep)
    
    ha = bb_candle_plotprep(def_ha(prep))
    
    source = ColumnDataSource(df)
    source_ha = ColumnDataSource(ha)
    
    green = CDSView(source=source, filters=[BooleanFilter(df.Open < df.Close)])
    red = CDSView(source=source, filters=[BooleanFilter(df.Open > df.Close)])


    green_ha = CDSView(source=source_ha, filters=[BooleanFilter(ha.Open < ha.Close)])
    red_ha = CDSView(source=source_ha, filters=[BooleanFilter(ha.Open > ha.Close)])
    
    hover = HoverTool(tooltips= [("Date", "@Date{%F}"), ("Open", "@Open"), ("High", "@High"),
                                 ("Low", "@Low"), ("Close", "@Close"), ("Upper", "@Upper"),
                                     ("Middle", "@Middle"), ("Lower", "@Lower")],
                                mode = "vline", formatters = {"Date": "datetime"},
                     names = ["foo", "boo"])
    
    

    
    p1 = figure(title = f_title, x_axis_type = "datetime", x_axis_label = "Date",
                y_axis_label = "Price",plot_width=750, tools = [hover,"crosshair","pan,box_zoom,wheel_zoom,save,reset,help"])
    p1.line(x = "Date", y = "Upper", source = source, color = "black")
    p1.line(x = "Date", y = "Middle", source = source, color = "black")
    p1.line(x = "Date", y = "Lower", source = source, color = "black")
    
   
    p1.segment('Date', "High", "Date", "Low", color="black", source = source)
    p1.vbar(x = "Date", top = "Close", bottom = 'Open', width = w, fill_color = "green", source = source,
            line_color = "black", view = green, name = "foo", hover_color = hex_color_green)
    p1.vbar("Date", w, "Open", 'Close', fill_color = "red", source = source,
            line_color = "black", view = red, name = "boo", hover_color = hex_color_red)
    
    
    
    p2 = figure(title = f_title + " Heikin-Ashi", x_axis_type = "datetime", x_axis_label = "Date",
                y_axis_label = "Price", plot_width=750, tools = [hover,"crosshair","pan,box_zoom,wheel_zoom,save,reset,help"])
    p2.line(x = "Date", y = "Upper", source = source_ha, color = "black")
    p2.line(x = "Date", y = "Middle", source = source_ha, color = "black")
    p2.line(x = "Date", y = "Lower", source = source_ha, color = "black")
    
   
    p2.segment('Date', "High", "Date", "Low", color="black", source = source_ha)
    p2.vbar(x = "Date", top = "Close", bottom = 'Open', width = w, fill_color =  "green", source = source_ha,
             line_color = "black",view = green_ha, name = "foo", hover_color = hex_color_green)
    p2.vbar("Date", w, "Open", 'Close', fill_color = "red", source = source_ha,
            line_color = "black", view = red_ha, name = "boo", hover_color = hex_color_red)
    
    p1.x_range = p2.x_range
    p1.y_range = p2.y_range
    
    """hover_color = hex_color_green
    hover_color = hex_color_red"""
    
    return p1, p2

In [6]:
def aroon(df, period = 6):
    aroon = pd.DataFrame().reindex(df.index)
    #aroon.index = df.index
    period = 6
    h_n_day = df["High"].rolling(period, min_periods = period).apply(np.argmax, raw = True)
    l_n_day = df["Low"].rolling(period, min_periods = period).apply(np.argmin, raw = True)
    aroon["Up"] = ((period - h_n_day)/ period) * 100
    aroon["Down"] = ((period - l_n_day)/ period) * 100
    aroon["Date"] = df["Date"]
    return aroon

In [7]:
def aroon_plot(file, period = 6):
    prep = dataprep(file)
    
    df = aroon(prep, period)
    ha = aroon(def_ha(prep), period)
    
    source = ColumnDataSource(df)
    source_ha = ColumnDataSource(ha)
    
    hover = HoverTool(tooltips = [("Date", "@Date{%F}"), ("Up", "@Up"), ("Down", "@Down")], formatters = {"Date": "datetime"}, mode = "vline", names = ["foo"])
    
    p1 = figure(x_axis_type = "datetime", plot_height = 120 , plot_width = 750,
                tools = [hover,"crosshair","pan,box_zoom,wheel_zoom,save,reset,help"])
    p1.line(y = "Up", x = "Date", color = "green", source = source, name = "foo")
    p1.line(y = "Down", x = "Date", color = "red", source = source)
    
    
    p2 = figure(x_axis_type = "datetime", plot_height = 120 , plot_width = 750,
               tools = [hover,"crosshair","pan,box_zoom,wheel_zoom,save,reset,help"])
    p2.line(y = "Up", x = "Date", color = "green", source = source_ha, name = "foo")
    p2.line(y = "Down", x = "Date", color = "red", source = source_ha)

    
    return p1, p2

In [8]:
def macd(df, macd1 = 26, macd2 = 12, macd3 = 9):
    macd = pd.DataFrame().reindex(df.index)
    df_mean = df.iloc[:, [1, 2, 3, 4]].mean(axis = 1)
    #macd1, macd2, macd3 = 26, 12, 9

    macd1_ema = df_mean.ewm(span = macd1, adjust = False).mean()
    macd2_ema = df_mean.ewm(span = macd2, adjust = False).mean()
    macd['MACD Line'] = macd2_ema - macd1_ema
    macd['Signal Line'] = macd[['MACD Line']].ewm(span = macd3, adjust = False).mean()
    macd["Histogram"] = macd['MACD Line'] - macd['Signal Line']
    macd["Date"] = df["Date"]
    return macd

In [9]:
def overlap_macd_plot(file, w = 12*60*60*1000):
    prep = dataprep(file)
    df = macd(prep)
    ha = macd(def_ha(prep))
    
    source = ColumnDataSource(df)
    source_ha = ColumnDataSource(ha)
    
    hover = HoverTool(tooltips = [("Date", "@Date{%F}"), ("MACD Line", "@{MACD Line}"),
                                  ("Signal Line", "@{Signal Line}"), ("Histogram", "@Histogram")],
                      formatters = {"Date": "datetime"}, names = ["foo"],
                     mode = "vline")
    
    p1 = figure(x_axis_type = "datetime", tools = [hover,"crosshair","pan,box_zoom,wheel_zoom,save,reset,help"],
               plot_height = 100, plot_width = 750)
    
    p1.line(x = 'Date', y = "MACD Line", color = "black", source = source, name = "foo")
    p1.line(x = 'Date', y = "Signal Line", color = "red", source = source)
    p1.vbar(x = "Date", top = "Histogram", bottom = 0, width = w, color = "black", alpha = .2, source = source)

    
    p2 = figure(x_axis_type = "datetime", tools = [hover,"crosshair","pan,box_zoom,wheel_zoom,save,reset,help"],
               plot_height = 100, plot_width = 750)
    
    p2.line(x = 'Date', y = "MACD Line", color = "black", source = source_ha, name = "foo")
    p2.line(x = 'Date', y = "Signal Line", color = "red", source = source_ha)
    p2.vbar(x = "Date", top = "Histogram", bottom = 0, width = w, color = "black", alpha = .2, source = source_ha)

    
    p1.x_range = p2.x_range
    p1.y_range = p2.y_range
    
    return p1, p2  

In [10]:
data = "AAPL5.csv" # Five yrs of Daily historical Data downloaded from Yahoo Finance
r, ha = reg_ha_bb_candle_plot(data)
r1, r2 = aroon_plot(data)
m1, m2 = overlap_macd_plot(data)
r.x_range = ha.x_range = r1.x_range = r2.x_range = m1.x_range = m2.x_range
save_layout = gridplot([[r, ha], [r1, r2], [m1, m2]])
show(save_layout) #Will open chart in a new window